Introducing/ Installing Pysyft

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0730 19:25:48.399705  6096 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0730 19:25:48.416702  6096 deprecation_wrapper.py:119] From C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Basic Remote Execution in Pysyft

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x=x.send(bob)

In [12]:
bob._objects

{41647280302: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

41647280302

In [15]:
x.id

85279586515

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:85279586515 -> bob:41647280302]

In [19]:
x = x.get()

In [20]:
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

Playing with remote tensor

In [23]:
alice = sy.VirtualWorker(hook, id ="alice")

In [24]:
x = th.tensor([1,2,3,4,5])

In [25]:
x_ptr = x.send(bob,alice)

In [26]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [28]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

Introducing Remote Arithmetic

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [30]:
x

(Wrapper)>[PointerTensor | me:11916196594 -> bob:7460496349]

In [31]:
z = x + y

In [32]:
z

(Wrapper)>[PointerTensor | me:67906244819 -> bob:67198481763]

In [33]:
z = z.get()

In [34]:
z

tensor([2, 3, 4, 5, 6])

In [40]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [41]:
z = (x+y).sum()

In [42]:
z.backward()

(Wrapper)>[PointerTensor | me:16542351276 -> bob:46833501868]

In [43]:
x = x.get()

In [44]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [45]:
x.grad

tensor([1., 1., 1., 1., 1.])

Learn a Simple Linear Model

In [46]:
input = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [47]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [48]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward() #backpropagate
    
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


Garbage Collection and common Errors

In [49]:
bob = bob.clear_objects()

In [51]:
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{56700496380: tensor([1, 2, 3, 4, 5])}

In [54]:
del x

In [55]:
bob._objects

{}

In [56]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
bob._objects

{75189826502: tensor([1, 2, 3, 4, 5])}

In [60]:
x = "asdf"

In [62]:
bob._objects

{}

In [63]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [64]:
x

(Wrapper)>[PointerTensor | me:90650883460 -> bob:57480523315]

In [65]:
bob._objects

{57480523315: tensor([1, 2, 3, 4, 5])}

In [66]:
x = "asdf"

In [67]:
bob._objects

{57480523315: tensor([1, 2, 3, 4, 5])}

In [68]:
del x

In [69]:
bob._objects

{57480523315: tensor([1, 2, 3, 4, 5])}

In [70]:
bob = bob.clear_objects()
bob._objects

{}

In [71]:
for i in range(1000):
    x =th.tensor([1,2,3,4,5]).send(bob)

In [72]:
bob._objects

{20459311670: tensor([1, 2, 3, 4, 5])}

In [73]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [74]:
z = x+y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:58362417379 -> bob:70065377479]
Tensor B: tensor([1, 1, 1, 1, 1])

In [75]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [76]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:48617885170 -> bob:82322307592]
Tensor B: tensor([1, 1, 1, 1, 1])

Toy Federated Learning

In [77]:
from torch import nn, optim

In [78]:
#A toy dataset
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [79]:
#A toy model
model = nn.Linear(2,1)

In [80]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [81]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()
        
        pred = model(data)
        
        loss = ((pred - target)**2).sum()
        
        loss.backward()
        
        opt.step()
        
        print(loss.data)
        
train()

tensor(2.7020)
tensor(1.0694)
tensor(0.7184)
tensor(0.5263)
tensor(0.3908)
tensor(0.2916)
tensor(0.2183)
tensor(0.1640)
tensor(0.1235)
tensor(0.0932)
tensor(0.0705)
tensor(0.0534)
tensor(0.0405)
tensor(0.0308)
tensor(0.0234)
tensor(0.0178)
tensor(0.0136)
tensor(0.0103)
tensor(0.0079)
tensor(0.0060)


In [93]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [94]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [90]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [97]:
def train(iterations=20):
    
    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr = 0.1)
    
    for iter in range(iterations):
        for _data, _target in datasets:
            
            #send model to the data
            model = model.send(_data.location)
            
            #do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            
            #get smarter model back
            model = model.get()
            
            print(loss.get())

In [98]:
train()

tensor(2.3864, requires_grad=True)
tensor(0.7426, requires_grad=True)
tensor(0.3301, requires_grad=True)
tensor(0.5222, requires_grad=True)
tensor(0.1744, requires_grad=True)
tensor(0.3326, requires_grad=True)
tensor(0.1083, requires_grad=True)
tensor(0.2127, requires_grad=True)
tensor(0.0692, requires_grad=True)
tensor(0.1377, requires_grad=True)
tensor(0.0451, requires_grad=True)
tensor(0.0902, requires_grad=True)
tensor(0.0299, requires_grad=True)
tensor(0.0599, requires_grad=True)
tensor(0.0203, requires_grad=True)
tensor(0.0402, requires_grad=True)
tensor(0.0140, requires_grad=True)
tensor(0.0274, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0188, requires_grad=True)
tensor(0.0070, requires_grad=True)
tensor(0.0131, requires_grad=True)
tensor(0.0050, requires_grad=True)
tensor(0.0092, requires_grad=True)
tensor(0.0036, requires_grad=True)
tensor(0.0065, requires_grad=True)
tensor(0.0027, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0020, requi

Advanced Remote Execution Tools

In [100]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [101]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [102]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [103]:
x = x.send(alice)

In [105]:
bob._objects

{69726157384: tensor([1, 2, 3, 4, 5])}

In [106]:
alice._objects

{62862936286: (Wrapper)>[PointerTensor | alice:62862936286 -> bob:69726157384]}

In [107]:
y = x+x

In [108]:
y

(Wrapper)>[PointerTensor | me:956132459 -> alice:87232960017]

In [109]:
bob._objects

{69726157384: tensor([1, 2, 3, 4, 5]),
 75534449223: tensor([ 2,  4,  6,  8, 10])}

In [110]:
alice._objects

{62862936286: (Wrapper)>[PointerTensor | alice:62862936286 -> bob:69726157384],
 87232960017: (Wrapper)>[PointerTensor | alice:87232960017 -> bob:75534449223]}

In [111]:
jon = sy.VirtualWorker(hook, id="jon")

In [112]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [114]:
bob._objects

{95520966728: tensor([1, 2, 3, 4, 5])}

In [116]:
alice._objects

{71747448214: (Wrapper)>[PointerTensor | alice:71747448214 -> bob:95520966728]}

In [118]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [119]:
bob._objects

{}

In [120]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [122]:
bob._objects

{15904888690: tensor([1, 2, 3, 4, 5])}

In [123]:
alice._objects

{18118552604: (Wrapper)>[PointerTensor | alice:18118552604 -> bob:15904888690]}

In [124]:
del x

In [126]:
bob._objects

{}

In [128]:
alice._objects

{}

Pointer Chain Operations

In [131]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [132]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [133]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [134]:
x

(Wrapper)>[PointerTensor | me:23803848641 -> bob:13093835312]

In [135]:
bob._objects

{13093835312: tensor([1, 2, 3, 4, 5])}

In [136]:
alice._objects

{}

In [137]:
x.move(alice) #an order to get the tensor from bob

(Wrapper)>[PointerTensor | me:23803848641 -> alice:23803848641]

In [139]:
bob._objects #now bob objects is empty

{}

In [142]:
alice._objects #tensor moved from bob to alice

{23803848641: tensor([1, 2, 3, 4, 5])}

In [143]:
#Using remote.get

In [156]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [157]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [158]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [159]:
x

(Wrapper)>[PointerTensor | me:10857549781 -> alice:27612945824]

In [160]:
bob._objects

{54056500281: tensor([1, 2, 3, 4, 5])}

In [161]:
alice._objects

{27612945824: (Wrapper)>[PointerTensor | alice:27612945824 -> bob:54056500281]}

In [162]:
x.remote_get() #order to alice to get tensor from bob

(Wrapper)>[PointerTensor | me:10857549781 -> alice:27612945824]

In [163]:
bob._objects #now bob tensor moved to alice

{}

In [164]:
alice._objects #alice got the tensor

{27612945824: tensor([1, 2, 3, 4, 5])}